<a href="https://colab.research.google.com/github/palomovisk/CienciadeDados/blob/main/runningMPI_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!apt update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,526 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,939 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd

In [19]:
!apt install openmpi-bin openmpi-common libopenmpi-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-common is already the newest version (4.1.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
%%writefile CalculoPIsendandreceiv-mpi.c

//Paloma de Sousa Bezerra
//RA: 10376795

#include "mpi.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define MAXSIZE 1000000

//Funcao para calcular pi
double calcula_pi(double *data, int local_n) {
    double sum = 0.0;
    for (int i = 0; i < local_n; i++) {
        double factor = ((int)data[i] % 2 == 0) ? 1.0 : -1.0;
        sum += factor / (2 * data[i] + 1);
    }
    return sum;
}

int main(int argc, char **argv) {
    int myid, numprocs;
    int local_n, i, low, high;
    double myresult = 0.0, result = 0.0;
    double *data = NULL;
    double *my_data;

    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
    MPI_Comm_rank(MPI_COMM_WORLD, &myid);

    local_n = MAXSIZE / numprocs;
    low = myid * local_n;
    high = low + local_n;

    // Aloca memória para os dados locais
    my_data = (double *)malloc(local_n * sizeof(double));

    // Processo raiz gera os dados
    if (myid == 0) {
        data = (double *)malloc(MAXSIZE * sizeof(double));
        for (i = 0; i < MAXSIZE; i++) {
            data[i] = i;
        }
    }

    // Envia dados para os processos escravos
    if (myid == 0) {
        for (i = 1; i < numprocs; i++) {
            low = i * local_n;
            high = low + local_n;
            MPI_Send(&data[low], local_n, MPI_DOUBLE, i, 0, MPI_COMM_WORLD);
        }
        // Dados para o processo raiz
        for (i = 0; i < local_n; i++) {
            my_data[i] = data[i];
        }
    } else {
        // Processos escravos recebem os dados
        MPI_Recv(my_data, local_n, MPI_DOUBLE, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    }

    // Cada processo calcula sua parte da série
    myresult = calcula_pi(my_data, local_n);

    // Envia o resultado parcial para o processo raiz
    if (myid != 0) {
        MPI_Send(&myresult, 1, MPI_DOUBLE, 0, 0, MPI_COMM_WORLD);
    }

    // Processo raiz coleta os resultados parciais
    if (myid == 0) {
        result = myresult;
        for (i = 1; i < numprocs; i++) {
            MPI_Recv(&myresult, 1, MPI_DOUBLE, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            result += myresult;
        }
        double pi = 4.0 * result;
        printf("Calculo do Pi: %.15f\n", pi);

        // Libera memória alocada
        free(data);
    }

    // Libera memória alocada
    free(my_data);

    MPI_Finalize();
    return 0;
}

Writing CalculoPIsendandreceiv-mpi.c


In [6]:
!mpicc -o CalculoPIsendandreceiv-mpi CalculoPIsendandreceiv-mpi.c

In [7]:
!mpiexec  --allow-run-as-root --oversubscribe  -n 4 ./CalculoPIsendandreceiv-mpi

Calculo do Pi: 3.141591653589781
